In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
import pandas as pd
import re
import requests
import browser_cookie3
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Date, Float, Text
from sqlalchemy.orm import sessionmaker
import bs4
from bs4 import BeautifulSoup
import datetime
from dataclass import *
from db_search_parser import all_race
from db_race_parser import *
pd.set_option('display.max_columns', None)

In [2]:
FIREFOX_PROFILE = r"C:\Users\temp\AppData\Roaming\Mozilla\Firefox\Profiles\oi8lq9qu.forselenium"
options = Options()
options.add_argument('-profile')
options.add_argument(FIREFOX_PROFILE)
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(),options=options)

C:\Users\temp\AppData\Local\Temp\ipykernel_14156\1028671364.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(),options=options)


In [19]:
engine = create_engine('sqlite:///sample_db.sqlite3')
Base.metadata.create_all(bind=engine)
SessionClass = sessionmaker(engine)  # セッションを作るクラスを作成
session = SessionClass()

In [4]:
# レース一覧にアクセス
url=r"https://db.netkeiba.com//?pid=race_list&word=&start_year=1988&start_mon=none&end_year=none&end_mon=none&jyo[0]=01&jyo[1]=02&jyo[2]=03&jyo[3]=04&jyo[4]=05&jyo[5]=06&jyo[6]=07&jyo[7]=08&jyo[8]=09&jyo[9]=10&kyori_min=&kyori_max=&sort=date&list=100&page="
i=1
driver.get(url+str(i))
driver.implicitly_wait(10)

In [5]:
raceinfos = all_race(driver.page_source)

In [6]:
race = raceinfos[1]

In [7]:
driver.switch_to.new_window('tab')

In [8]:
driver.get(f"https://db.netkeiba.com/race/{race._id}")

In [9]:
race.set_direction(driver.page_source)
_id = race._id

In [10]:
races = get_race(driver.page_source, _id)
oddses = get_odds(driver.page_source, _id)
truck_infos = get_truck_info(driver.page_source, _id)
corner_infos = get_corner_info(driver.page_source, _id)
lap_times = get_lap_time(driver.page_source, _id)
race_analysis = get_race_analysis(driver.page_source, _id)
short_comment = get_short_comment(driver.page_source, _id)

In [20]:
session.add_all(races)
session.add_all(oddses)
if truck_infos:
    session.add(truck_infos)
if corner_infos:
    session.add(corner_infos)
if lap_times:
    session.add(lap_times)
if race_analysis:
    session.add(race_analysis)
if short_comment:
    session.add(short_comment)
    
session.add(race)
session.commit()

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.findAll("table")
race_table = table[0]
trs = race_table.find_all("tr")
tr = trs[1]

In [ ]:
driver.switch_to.new_window('tab')

In [ ]:
driver.close()

In [ ]:
x = Race()
x.from_tr(tr)
x.set_id(re.findall(r"\d{8,}", driver.current_url)[0])

In [ ]:
pay_block = soup.find(class_="pay_block")
trs = pay_block.find_all("tr")
res =[]
_id = re.findall(r"\d{8,}", driver.current_url)[0]
for tr in trs:
    kind = tr.find("th").text
    atari,henkin,ninki = tr.find_all("td")
    atari = atari.get_text('UNKO').split('UNKO')
    henkin = henkin.get_text('UNKO').split('UNKO')
    ninki = ninki.get_text('UNKO').split('UNKO')
    res += [Odds(_id,kind,a,h,n) for a,h,n in zip(atari,henkin,ninki)]

In [ ]:
# 馬場情報
result_info = soup.find_all(class_="result_info")
tables_left = result_info[0].find_all("table")
for table in tables_left:
    if re.search(r"馬場指数", table.text):
        baba_index = table
        break
xx=TruckInfo(_id, baba_index)

In [ ]:
# コーナー通過順位
result_info = soup.find_all(class_="result_info")
tables_left = result_info[0].find_all("table")
for table in tables_left:
    if re.search(r"コーナー通過順位", table.text):
        corner_index = table
        break
xxx = CornerInfo(_id, corner_index)

In [ ]:
# ラップタイム
result_info = soup.find_all(class_="result_info")
tables_left = result_info[0].find_all("table")
for table in tables_left:
    if re.search(r"ラップタイム", table.text):
        lap_time = table
        break
xxxx = LapTime(_id, lap_time)

In [ ]:
# レース分析
race_anal = soup.find_all(class_="result_info")
tables_right = result_info[1].find_all("table")
for table in tables_right:
    if re.search(r"レース分析", table.text):
        race_anal_table = table
        break
xxxxx = RaceAnalysis(_id, race_anal_table)
xxxxx.__dict__

In [ ]:
# レース後の短評
race_tanpyo = soup.find_all(class_="result_info")
tables_right = result_info[1].find_all("table")
for table in tables_right:
    if re.search(r"レース後の短評", table.text):
        race_tanpyo_table = table
        break

ths = race_tanpyo_table.find_all("th")
tds = race_tanpyo_table.find_all("td")
if ths is not None or tds is not None:
    x=10
[ShortComment().init_thtd(_id,th,td).__dict__ for th,td in zip(ths,tds)]